In [1]:
from pathlib import Path
READY = Path("../data/ready")
list(READY.glob("*_ready.parquet"))


[WindowsPath('../data/ready/HI_Medium_ready.parquet'),
 WindowsPath('../data/ready/HI_Small_ready.parquet'),
 WindowsPath('../data/ready/LI_Medium_ready.parquet'),
 WindowsPath('../data/ready/LI_Small_ready.parquet')]